In [ ]:
## Multi- Spectrum Fitting by Segment

import numpy as np
import numpy.ma as ma
import pandas as pd
import qgrid
import os
from lmfit import minimize, Parameters, report_fit, Model, Minimizer
from lmfit.models import GaussianModel, LinearModel, Model, VoigtModel
import matplotlib.pyplot as plt
from matplotlib import gridspec
from scipy import interpolate

import sys
sys.path.append(r'C:\Users\ema3\Documents\Python Scripts\HAPI')# set location of HAPI.py module
sys.path.append(r'C:\Users\ema3\Documents\Cold Cavity - O2 A Band')# set location of HAPI.py module
%matplotlib inline
from hapi import *
from hapi_HTP_definition_dataframe_190409 import *

import seaborn as sns
sns.set_style("whitegrid")
sns.set_style("ticks")
sns.set_context("poster")


In [ ]:
path = r'C:\Users\ema3\Documents\Data\Intensity Scans' # Location of the Summary Data File
os.chdir(path)

#Generic Fit Parameters
wave_range = 1.5 #range outside of experimental x-range to simulate
IntensityThreshold = 1e-30 #intensities must be above this value to be simulated
Fit_Intensity = 1e-24 #intensities must be above this value for the line to be fit
order_baseline_fit = 1
tau_column = 'Corrected Tau - Etalon' # Tau - Etalon, Corrected Tau - Etalon, Corrected Tau (us), Mean tau/us
#
freq_column = 'Frequency (FSR) /MHz' # Total Frequency /MHz, Total Frequency (Detuning)


#Define all Spectra individually
spec_1 = Spectrum('21Feb18_237.5K_0.2__1000Torr_etalon_Seg5', 
                        concentration = { 7 : 0.002022}, natural_abundance = False, diluent = 'air', 
                        etalons = {}, 
                        input_freq = True, frequency_column = freq_column,
                        input_tau = True, tau_column = tau_column, tau_stats_column = None, 
                        pressure_column = 'Cavity Pressure /Torr', temperature_column = 'Cavity Temperature Side 2 /C', 
                        nominal_temperature = 237, abundance_ratio_MI = {7:{1:1/0.995262, 2:0 ,3:0}}, x_shift = 0.00)
spec_2 = Spectrum('22Feb18_237.5K_0.2__700Torr_etalon_Seg5', 
                        concentration = { 7 : 0.002022}, natural_abundance = False, diluent = 'air', 
                        etalons = {}, 
                        input_freq = True, frequency_column = freq_column,
                        input_tau = True, tau_column = tau_column, tau_stats_column = None, 
                        pressure_column = 'Cavity Pressure /Torr', temperature_column = 'Cavity Temperature Side 2 /C', 
                        nominal_temperature = 237, abundance_ratio_MI = {7:{1:1/0.995262, 2:0 ,3:0}}, x_shift = 0.00)
spec_3 = Spectrum('23Feb18_237.5K_0.2__400Torr_etalon_Seg5', 
                        concentration = { 7 : 0.002022}, natural_abundance = False, diluent = 'air', 
                        etalons = {}, 
                        input_freq = True, frequency_column = freq_column,
                        input_tau = True, tau_column = tau_column, tau_stats_column = None, 
                        pressure_column = 'Cavity Pressure /Torr', temperature_column = 'Cavity Temperature Side 2 /C', 
                        nominal_temperature = 237, abundance_ratio_MI = {7:{1:1/0.995262, 2:0 ,3:0}}, x_shift = 0.00)




#Add all spectrum to a Dataset object
SPECTRA = Dataset([spec_1, spec_2, spec_3], 'Old Data', baseline_order = order_baseline_fit)

#Generate Baseline Parameter list based on number of etalons in spectra definitions and baseline order
BASE_LINELIST = SPECTRA.generate_baseline_paramlist()

#Read in Possible linelists
hapi = r'C:\Users\ema3\Documents\Python Scripts\HAPI'
os.chdir(hapi)
PARAM_LINELIST = pd.read_csv('Drouin_2017_linelist.csv')

print (SPECTRA.get_number_nominal_temperatures())

In [ ]:
os.chdir(path)
# Set-up for Fitting
#lineprofile = 'NGP' #VP, SDVP, NGP, SDNGP, HTP



        
        
        

FITPARAMS = Generate_FitParam_File(SPECTRA, PARAM_LINELIST, BASE_LINELIST, lineprofile = 'SDVP', linemixing = True, 
                                  fit_intensity = Fit_Intensity, threshold_intensity = IntensityThreshold, 
                                  nu_constrain = False, sw_constrain = True, gamma0_constrain = True, delta0_constrain = True, 
                                   aw_constrain = True, as_constrain = True, 
                                   nuVC_constrain = True, eta_constrain =True, linemixing_constrain = True)

FITPARAMS.generate_fit_param_linelist_from_linelist(vary_nu = {7:{1:True, 2:False, 3:False}}, vary_sw = {7:{1:True, 2:False, 3:False}},
                                                    vary_gamma0 = {7:{1: True, 2:False, 3: False}, 1:{1:False}}, vary_n_gamma0 = {7:{1:True}}, 
                                                    vary_delta0 = {7:{1: False, 2:False, 3: False}, 1:{1:False}}, vary_n_delta0 = {7:{1:True}}, 
                                                    vary_aw = {7:{1: True, 2:False, 3: False}, 1:{1:False}}, vary_n_gamma2 = {7:{1:False}}, 
                                                    vary_as = {}, vary_n_delta2 = {7:{1:False}}, 
                                                    vary_nuVC = {7:{1:False}}, vary_n_nuVC = {7:{1:False}},
                                                    vary_eta = {}, vary_linemixing = {7:{1:True}})

FITPARAMS.generate_fit_baseline_linelist(vary_baseline = True, vary_concentration = {7:True, 1:False}, vary_xshift = False, 
                                      vary_etalon_amp= False, vary_etalon_freq= False, vary_etalon_phase= False)





In [ ]:
edit_files = Edit_Fit_Param_Files('Baseline_LineList', 'Parameter_LineList')
edit_baseline = edit_files.edit_generated_baselist()
edit_baseline

In [ ]:
edit_params = edit_files.edit_generated_paramlist()
edit_params

In [ ]:
edit_files.save_editted_baselist(edit_baseline)
edit_files.save_editted_paramlist(edit_params)
print ('')

In [ ]:
os.chdir(path)   
fit_data = Fit_DataSet(SPECTRA,'Baseline_LineList', 'Parameter_LineList', fit_intensity = Fit_Intensity, 
                baseline_limit = False, baseline_limit_percent = 10, 
                concentration_limit = True, concentration_limit_percent = 1.1, 
                etalon_limit = False, etalon_limit_percent = 2, #phase is constrained to +/- 2pi, 
                x_shift_limit = True, x_shift_limit_magnitude = 0.5, 
                nu_limit = False, nu_limit_magnitude = 1, 
                sw_limit = True, sw_limit_percent = 2, 
                gamma0_limit = True, gamma0_limit_percent = 3, n_gamma0_limit= False, n_gamma0_limit_percent = 50, 
                delta0_limit = True, delta0_limit_percent = 2, n_delta0_limit = False, n_delta0_limit_percent = 50, 
                SD_gamma_limit = True, SD_gamma_limit_percent = 2, n_gamma2_limit = False, n_gamma2_limit_percent = 50, 
                SD_delta_limit = True, SD_delta_limit_percent = 50, n_delta2_limit = False, n_delta2_limit_percent = 50, 
                nuVC_limit = False, nuVC_limit_percent = 2, n_nuVC_limit = False, n_nuVC_limit_percent = 50, 
                eta_limit = True, eta_limit_percent = 50, linemixing_limit = False, linemixing_limit_percent = 50, constrain_n_SD = True)
params = fit_data.generate_params()
#params['etalon_2_freq_1'].set(expr='2*etalon_1_freq_1')
#params['etalon_2_phase_1'].set(expr='etalon_1_phase_1')


for param in params:
    if 'n_gamma2' in param:
        newparam = param.replace('n_gamma2', 'n_gamma0')
        params[param].set(expr = newparam)



#print (params)

#add_expr_params(self, params, param_name, expr)

result = fit_data.fit_data(params, wave_space = 1e-3, wing_cutoff = 25, wing_wavenumbers = 25)
print (result.params.pretty_print())

fit_data.residual_analysis(result, indv_resid_plot=True)
fit_data.update_params(result)




In [ ]:
spec_2.fft_spectrum()


s_list = [spec_1, spec_2, spec_3, spec_4, spec_5, spec_6, spec_7, spec_8, spec_9, spec_10, spec_11, spec_12, spec_13, spec_14, spec_15, spec_16, 
         spec_17, spec_18, spec_19, spec_20, spec_21, spec_22, spec_23, spec_24]
for s in s_list:
    
    print (s.get_pressure_torr(), s.get_temperature_C())



